In [1]:
import numpy as np
import pandas as pd
import pystan
from codebase.file_utils import save_obj, load_obj
from codebase.data import get_data


%load_ext autoreload
%autoreload 2

In [2]:
data = get_data(50, 6, 1)
data

{'random_seed': 1,
 'N': 50,
 'J': 6,
 'alpha': array([0., 0., 0., 0., 0., 0.]),
 'Phi_cov': array([[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]]),
 'z': array([[ 1.62434536e+00, -6.11756414e-01, -5.28171752e-01,
         -1.07296862e+00,  8.65407629e-01, -2.30153870e+00],
        [ 1.74481176e+00, -7.61206901e-01,  3.19039096e-01,
         -2.49370375e-01,  1.46210794e+00, -2.06014071e+00],
        [-3.22417204e-01, -3.84054355e-01,  1.13376944e+00,
         -1.09989127e+00, -1.72428208e-01, -8.77858418e-01],
        [ 4.22137467e-02,  5.82815214e-01, -1.10061918e+00,
          1.14472371e+00,  9.01590721e-01,  5.02494339e-01],
        [ 9.00855949e-01, -6.83727859e-01, -1.22890226e-01,
         -9.35769434e-01, -2.67888080e-01,  5.30355467e-01],
        [-6.91660752e-01, -3.96753527e-01, -6.87172700e-01,
         -8.45205641e-01, -6.712

In [14]:

path_to_stan = './codebase/stancode/'

with open(path_to_stan+'logit0.stan', 'r') as file:
    model_code = file.read()

param_names = ['alpha', 'Marg_cov', 'z']

sm = pystan.StanModel(model_code=model_code, verbose=False)
# save_obj(sm, 'sm', log_dir)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_3b8a020f9e49fab3e7adf8f6613c97f2 NOW.


In [15]:
log_dir = './log/demo/'
save_obj(sm, 'sm', log_dir)


In [3]:

num_samples = 1000
num_warmup = 1000
num_chains = 1
fit_run = sm.sampling(
    data={
        'N':data['N'],
        'J': data['J'],
        'y' : data['y']
    },
    iter=num_samples + num_warmup,
    warmup=num_warmup,
    chains=num_chains,
#     algorithm = 'Fixed_param',
    n_jobs=1
)
samples = fit_run.extract(
    permuted=False, pars=param_names)

save_obj(samples, 'ps', log_dir)

NameError: name 'sm' is not defined

In [5]:

path_to_stan = './codebase/stancode/'

with open(path_to_stan+'logit_0_prior.stan', 'r') as file:
    model_code = file.read()

param_names = ['alpha', 'Marg_cov', 'z']

sm = pystan.StanModel(model_code=model_code, verbose=False)

num_samples = 100
num_warmup = 0
num_chains = 1
fit_run = sm.sampling(
    data={
        'N':data['N'],
        'J': data['J'],
        'y' : data['y']
    },
    iter=num_samples + num_warmup,
    warmup=num_warmup,
    chains=num_chains,
    algorithm = 'Fixed_param',
    n_jobs=1
)
prior_samples = fit_run.extract(
    permuted=False, pars=param_names)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_36e41c22590d83dfc4e02c60341018ba NOW.


In [9]:
np.round(prior_samples['Marg_cov'][0],2)

array([[[ 1.  , -0.1 , -0.33,  0.39,  0.25,  0.33],
        [-0.1 ,  1.  , -0.13, -0.09, -0.43, -0.09],
        [-0.33, -0.13,  1.  ,  0.03,  0.5 , -0.13],
        [ 0.39, -0.09,  0.03,  1.  , -0.16,  0.28],
        [ 0.25, -0.43,  0.5 , -0.16,  1.  , -0.26],
        [ 0.33, -0.09, -0.13,  0.28, -0.26,  1.  ]]])

In [ ]:
particles = sample_prior_particles()

In [ ]:
degeneracy_limit = .5
for i in range(1,nsim_data):
    data_temp["N"] = int(i+1)
    data_temp['y'] = data['y'][i].copy()


    log_incr_weights = get_weights(data_temp, particles)
    loglkhds[i] = logsumexp(log_weights + log_incr_weights) - logsumexp(log_weights)
    log_weights = log_incr_weights + log_weights
    print("ESS: %.2f"%(ESS(log_weights)))
    
    if (ESS(log_weights) < degeneracy_limit * nsim_prtcls) and (i+1) < nsim_data:
        data_temp['y'] = data['y'][:(i+1)].copy()
        print("Deg %d"%(i))
        particles = multinomial_sample_particles(particles, np.exp(log_weights))
    
        particles = jitter(data_temp, particles)
        log_weights = np.zeros(nsim_prtcls)
    else:
        pass
